In [1]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.2 MB/s eta 0:00:00


In [2]:
pip install -U langchain huggingface-hub

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the updated HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Use embeddings
text = "This is a test sentence."
vector = embeddings.embed_query(text)
print(vector)


<ipython-input-3-cf006b978124>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[0.08429647237062454, 0.057953689247369766, 0.004493385087698698, 0.10582111030817032, 0.007083410397171974, -0.017844678834080696, -0.016888074576854706, -0.015228300355374813, 0.0404730923473835, 0.033422548323869705, 0.10432765632867813, -0.04703585058450699, 0.006884715985506773, 0.04101794958114624, 0.01871197484433651, -0.04149235039949417, 0.02364744432270527, -0.056501854211091995, -0.03369623050093651, 0.05099096521735191, 0.06930321455001831, 0.05478416383266449, -0.009788415394723415, 0.02369721420109272, 0.019996633753180504, 0.00971734058111906, -0.0588991604745388, 0.0073074172250926495, 0.04702650383114815, -0.0045100972056388855, -0.05579973757266998, -0.004159417934715748, 0.06475705653429031, 0.04807629436254501, 0.017020823433995247, -0.00318335834890604, 0.057402364909648895, 0.035231854766607285, -0.005883913021534681, 0.014832933433353901, 0.011576307937502861, -0.1074807345867157, 0.019104203209280968, 0.022085724398493767, 0.01086454652249813, 0.0037819365970790

In [7]:
import pandas as pd

# Load the dataset
data = pd.read_csv("dataset (1).csv")

# Select more relevant columns for diversified descriptions
columns_to_include = [
    'Id', 'MSZoning', 'LotArea', 'Street', 'SalePrice',
    'YearBuilt', 'LotConfig', 'Condition1', 'BldgType',
    'HouseStyle', 'OverallQual', 'OverallCond', 'RoofStyle'
]
subset = data[columns_to_include].copy()

# Create descriptions using the additional columns
subset['Description'] = subset.apply(
    lambda row: (
        f"Property ID: {row['Id']} is a {row['HouseStyle']} style {row['BldgType']} "
        f"located in a {row['LotConfig']} lot configuration. It is zoned as {row['MSZoning']} "
        f"with a lot area of {row['LotArea']} sqft on a {row['Street']} street. "
        f"The property was built in {row['YearBuilt']} and has an overall quality rating of {row['OverallQual']} "
        f"and condition rating of {row['OverallCond']}. It features a {row['RoofStyle']} roof and is priced at ${row['SalePrice']}. "
        f"The property is described as {row['Condition1']}."
    ),
    axis=1
)

# Convert descriptions into a list for use in the RAG pipeline
descriptions = subset['Description'].tolist()

# Display a few descriptions to verify
for i, description in enumerate(descriptions[:5], 1):
    print(f"{i}: {description}")

1: Property ID: 1 is a 2Story style 1Fam located in a Inside lot configuration. It is zoned as RL with a lot area of 8450 sqft on a Pave street. The property was built in 2003 and has an overall quality rating of 7 and condition rating of 5. It features a Gable roof and is priced at $208500. The property is described as Norm.
2: Property ID: 2 is a 1Story style 1Fam located in a FR2 lot configuration. It is zoned as RL with a lot area of 9600 sqft on a Pave street. The property was built in 1976 and has an overall quality rating of 6 and condition rating of 8. It features a Gable roof and is priced at $181500. The property is described as Feedr.
3: Property ID: 3 is a 2Story style 1Fam located in a Inside lot configuration. It is zoned as RL with a lot area of 11250 sqft on a Pave street. The property was built in 2001 and has an overall quality rating of 7 and condition rating of 5. It features a Gable roof and is priced at $223500. The property is described as Norm.
4: Property ID: 4

In [8]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 52.5 MB/s eta 0:00:00


In [9]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Generate embeddings and set up FAISS retriever
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_texts(descriptions, embeddings)
retriever = vector_store.as_retriever()

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline

# Load a text-generation model
generator_tokenizer = AutoTokenizer.from_pretrained("gpt2")
generator_model = AutoModelForCausalLM.from_pretrained("gpt2")
generator_pipeline = pipeline("text-generation", model=generator_model, tokenizer=generator_tokenizer)

# Wrap the generator in LangChain
llm = HuggingFacePipeline(pipeline=generator_pipeline)



tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-9-6d24247de2e5>:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generator_pipeline)


In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load HuggingFace's text generation model
generator_tokenizer = AutoTokenizer.from_pretrained("gpt2")
generator_model = AutoModelForCausalLM.from_pretrained("gpt2")

# Set up the text-generation pipeline with adjusted parameters
generator_pipeline = pipeline(
    "text-generation",
    model=generator_model,
    tokenizer=generator_tokenizer,
    max_new_tokens=200,  # Allow the model to generate up to 200 tokens
    pad_token_id=generator_tokenizer.eos_token_id  # Avoid padding errors
)


Device set to use cuda:0


In [31]:
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Wrap the generator in LangChain's LLM interface
llm = HuggingFacePipeline(pipeline=generator_pipeline)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Do not include unrelated information.\n\n"
        "Question: {question}\n\n"
        "Answer:"
    )
)

# Create the QA chain with the retriever and updated generator
def ask_question(query):
    """
    Handles the query by retrieving relevant context and generating an answer.

    Args:
        query (str): The user's question.

    Returns:
        None: Prints the retrieved context and the generated answer.
    """
    try:
        # Invoke the QA chain with the query
        response = qa_chain.invoke({"query": query})

        # Retrieve the most relevant document's content
        if response.get('source_documents'):
            retrieved_context = response['source_documents'][0].page_content
            print(f"Retrieved Context:\n{retrieved_context}\n")
        else:
            retrieved_context = "No relevant context found."
            print("Retrieved Context: None\n")

        # Print the generated answer

    except Exception as e:
            retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})


In [34]:
import ipywidgets as widgets
from IPython.display import display

query_input = widgets.Text(
    placeholder="Ask your question here...",
    description="Query:",
    layout=widgets.Layout(width="80%")
)

output = widgets.Output()

query_input.on_submit(on_submit)
display(query_input, output)


Text(value='', description='Query:', layout=Layout(width='80%'), placeholder='Ask your question here...')

Output()

In [35]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00
